In [1]:
import arcpy
# from osgeo import gdal
# from osgeo import ogr
import pandas as pd
import os
import logging
import datetime

logging.basicConfig(level=logging.DEBUG)

today = datetime.date.today().strftime("%Y-%m-%d")



w_m_codeblock="""def cat (m, d):
    if m <=3:
        return 'Winter'
    elif m == 12 and d >=15:
        return 'Winter'
    elif m == 11:
        return 'Movement'
    elif m == 12 and d <15:
        return 'Movement'
    elif m ==4 or m ==5:
        return 'Movement'"""


In [ ]:
#existing data to append to
current_wrkspc=r'T:\\BHS_Test_3.gdb'
e_winter_pre=fr"{current_wrkspc}/BHS_Winter_Pre_1998_Survey"
e_winter_post=fr"{current_wrkspc}/BHS_Winter_Post_1998_Survey"
e_movement_pre=fr"{current_wrkspc}/BHS_Movement_Pre_1998_Survey"
e_movement_post=fr"{current_wrkspc}/BHS_Movement_Post_1998_Survey"

#input data, value to dictionary must be either Survey or Telemetry
inp_1={r'T:\merge_test.gdb\EVW_BHS_Collar_Nov_23':'Telemetry'}
inp_2={r'T:\EVW_BHS_collar_locations_13Nov23.shp':'Telemetry'}
inp_3= {None:None}
inp_4={None:None}
#maybe just make the entire thing a dictionary and not a list 
new_fcs=[inp_1, inp_2,inp_3,inp_4]

# print(new_fcs)

temp_out=r'T:\bhs_temp_data.gdb'

if arcpy.Exists(temp_out):
    print('gdb exists')
else:
    arcpy.management.CreateFileGDB(r'T:','bhs_temp_data.gdb')
    print('gdb created')
arcpy.env.workspace=temp_out

In [ ]:
#sort features into categories 
#Pre/post, winter/movement 

for fc in new_fcs:
    for key in fc:
        if key is not None:
            spatial_ref = arcpy.Describe(key).spatialReference
            print(spatial_ref.name)
            #check sptial ref
            fc_str=os.path.basename(key)
            fc_str=fc_str.split('.')[0]+'_BC_Albers'
            print(fc_str)
            if spatial_ref.name != 'NAD_1983_BC_Environment_Albers' and not arcpy.Exists(fc_str):
                arcpy.management.Project(fc, fc_str, 3005)
                key=fc_str
            elif arcpy.Exists(fc_str):
                print('albers fc exists')
                key=fc_str

            year_field=arcpy.ListFields(key, '*Year*')
            month_field=arcpy.ListFields(key, '*Month*')
            day_field= arcpy.ListFields(key, '*Day*')
            date_field=arcpy.ListFields(key, '*Date*')
            
            if year_field[0] is not None:
                print(f" year field name is {year_field[0].name}")
                print("f year field type is {year_field[0].type}")
                year_field=year_field[0].name
                before_1998=arcpy.management.SelectLayerByAttribute(key,'NEW_SELECTION',f"{year_field} < 1998")

            if month_field[0] is not None:
                print(f" year field name is {month_field[0].name}")
                print(f" year field type is {month_field[0].type}")
                month_field=month_field[0].name

            if date_field[0] is not None:
                #check for date field and split to get year type could be text, or datetime?
                print(f" date field name is {date_field[0].name}")
                print(f" date field type is {date_field[0].type}")
                date_field=date_field[0].name

            if year_field[0] is None and date_field[0] is not None:
                year_field='Year'
                arcpy.management.AddField(key, 'Year', 'DOUBLE')
                arcpy.management.CalculateField(key,'Year',f"Year($feature.{date_field[0].name})", "ARCADE" )
                before_1998=arcpy.management.SelectLayerByAttribute(key,'NEW_SELECTION',"Year < 1998")

            else:
                #no year field, check data!
                print('NO YEAR OR DATE FIELD CHECK DATA')
                exit()
            
            if day_field[0] is not None:
                print(f" day field name is {month_field[0].name}")
                print(f" day field type is {month_field[0].type}")
                day_field=day_field[0].name
            else:
                day_field='Day'
                arcpy.management.AddField(key, 'Day', 'DOUBLE')
                arcpy.management.CalculateField(key,'Year',f"Day($feature.{date_field[0].name})", "ARCADE" )

            before_1998_count=int(arcpy.GetCount_management (before_1998).getOutput (0))

            winter_pre="Year < 1998 And Cat ='Winter'"
            winter_pre_name=fr"{e_winter_pre}_{fc[key]}"

            winter_post="Year >= 1998 And Cat ='Winter'"
            winter_post_name=fr"{e_winter_post}_{fc[key]}"

            movement_pre="Year < 1998 And Cat ='Movement'" 
            movement_pre_name=fr"{e_movement_pre}_{fc[key]}"

            movement_post="Year < 1998 And Cat >='Movement'" 
            movement_post_name=fr"{e_movement_post}_{fc[key]}"          
            
            e_movement_post=fr"{current_wrkspc}/BHS_Movement_Post_1998_Survey"

            if before_1998_count >0:
                arcpy.management.AddField(key, 'Cat', 'TEXT')
                arcpy.management.CalculateField(key, 'Cat', f"cat (!{month_field}!, !{day_field}!)",'PYTHON3',w_m_codeblock)


                #-----------------Need to check for overlaps and duplicate data before appending-------------------------
                #-----------------Need to check for overlaps and duplicate data before appending-------------------------      
                #-----------------Need to check for overlaps and duplicate data before appending-------------------------      
                #-----------------Need to check for overlaps and duplicate data before appending-------------------------     


                #append new data?
                fieldMappings = arcpy.FieldMappings()
                fieldMappings.addTable(winter_pre_name)
                fieldMappings.addTable(key)
                arcpy.management.Append(key, winter_pre_name, schema_type= "NO_TEST ", field_mapping=fieldMappings, expression=winter_pre)

                fieldMappings = arcpy.FieldMappings()
                fieldMappings.addTable(winter_post_name)
                fieldMappings.addTable(key)
                arcpy.management.Append(key, winter_post_name, schema_type= "NO_TEST ", field_mapping=fieldMappings, expression=winter_post)         

                fieldMappings = arcpy.FieldMappings()
                fieldMappings.addTable(movement_pre_name)
                fieldMappings.addTable(key)
                arcpy.management.Append(key, movement_pre_name, schema_type= "NO_TEST ", field_mapping=fieldMappings, expression=movement_pre)

                fieldMappings = arcpy.FieldMappings()
                fieldMappings.addTable(movement_post_name)
                fieldMappings.addTable(key)
                arcpy.management.Append(key, movement_post_name, schema_type= "NO_TEST ", field_mapping=fieldMappings, expression=movement_post)                          
                # get pre winter and move layers
            
            
            
            else:
                arcpy.management.AddField(fc, 'Cat', 'TEXT')
                arcpy.management.CalculateField(fc, 'Cat', f"cat (!{month_field}!, !{day_field}!)",'PYTHON3',w_m_codeblock)
                
                fieldMappings = arcpy.FieldMappings()
                fieldMappings.addTable(winter_post_name)
                fieldMappings.addTable(key)
                arcpy.management.Append(key, winter_post_name, schema_type= "NO_TEST ", field_mapping=fieldMappings, expression=winter_post)       

                fieldMappings = arcpy.FieldMappings()
                fieldMappings.addTable(movement_post_name)
                fieldMappings.addTable(key)
                arcpy.management.Append(key, movement_post_name, schema_type= "NO_TEST ", field_mapping=fieldMappings, expression=movement_post)     


In [ ]:
#compare feautres for duplicates

In [ ]:
#compare attributes and preform mapping if needed 

In [ ]:
#merge features